In [ ]:
%pip install doctran
%pip install qdrant
%pip install sentence-transformers
%pip install pypdf
%pip install spacy

In [ ]:
!python -m spacy download  en_core_web_sm

In [20]:
from doctran import Doctran
from qdrant_client import QdrantClient

In [21]:
client = QdrantClient(":memory:")

In [ ]:
from pypdf import PdfReader

In [22]:
from pypdf import PdfReader
import io
import tqdm as notebook_tqdm

pages = []
with open("./api/association.pdf", "rb") as f:
    content = f.read()
    reader = PdfReader(io.BytesIO(content))
    for page in reader.pages:
        pages.append(page.extract_text())

In [ ]:
from sentence_transformers import SentenceTransformer
import spacy

nlp = spacy.load("en_core_web_sm")

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
for page in pages:
    doc = nlp(page)
    sentences = [s.text for s in doc.sents]
    embeddings = model.encode(sentences)
    print(f"{len(embeddings)} for {len(sentences)}")


In [23]:
from qdrant_client.models import VectorParams, Distance

client.create_collection("Notebook", vectors_config= VectorParams(size=model.get_sentence_embedding_dimension(),distance=Distance.COSINE))

In [31]:
from sentence_transformers import SentenceTransformer
import spacy
from qdrant_client.http.models import PointStruct

nlp = spacy.load("en_core_web_sm")

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
for page in pages:
    doc = nlp(page)
    sentences = [s.text for s in doc.sents]
    embeddings = model.encode(sentences)
    client.upsert(collection_name="Notebook", points = [ PointStruct(id=i,vector=embeddings[i].tolist(),payload={"sent":sent}) for i,sent in enumerate(sentences)])
    print(f"{len(embeddings)} for {len(sentences)}")



58 for 58
44 for 44
23 for 23
17 for 17
1 for 1
16 for 16
12 for 12
11 for 11
8 for 8
16 for 16
3 for 3
16 for 16
11 for 11
13 for 13
4 for 4


In [34]:
embeddings = model.encode(["What is an association?"])
results = client.search("Notebook", embeddings[0].tolist())
print(f"{len(results)} for {results}")

1 for [ScoredPoint(id=0, version=0, score=0.12582741132410996, payload={'sent': 'KAL Order 1973 - 1.11.1973 \n \nAmendments (Section-wise) \n \nSections  Act No. and year Remarks \nTitle, Preamble KAL Order 1973 w.e.f. 1.11.1973 \n1(1)(2) KAL Order 1973 w.e.f. 1.11.1973 \n \n \n \n '}, vector=None)]


In [35]:
client.get_collection("Notebook")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=1, indexed_vectors_count=0, points_count=1, segments_count=1, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=None, replication_factor=None, write_consistency_factor=None, on_disk_payload=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})